In [150]:
import numpy as np
from PIL import Image
import os
from matplotlib import pyplot as plt
%matplotlib inline

In [151]:
K = np.arange(26,27)
#K=10

In [160]:
img = Image.open("img/test06.jpg")
#img = img.resize((int(256*1.5),256))
img_width, img_height = img.size
imgarr = np.asarray(img)

In [161]:
def getcentroids(centroids,clusters):
    k = len(centroids)
    new_centroids = np.ndarray(shape=(k,3))
    for i in range(k):
        icluster= imgarr[np.where(clusters==i)]
        new_centroids[i] = np.mean(icluster, axis=0)        
    return new_centroids

In [162]:
def getclusters(centroids):
    k=len(centroids)    
    distances = np.ndarray(shape=((k,)+np.shape(imgarr)[:-1]))
    for i in range(k):
        distances[i] = np.sqrt(np.sum((imgarr-centroids[i])**2,axis=2))
    clusters = np.argmin(distances, axis = 0)
    return clusters    

In [163]:
def converged(centroids,old_centroids):
    threshold = 0.1
    val = np.sum((centroids-old_centroids)**2)
    if val<threshold:
        return True
    else:
        return False    

In [164]:
def getsegmentation(centroids):
    img = Image.new('RGB', (img_width, img_height), "white")
    img = np.asarray(img)
    clusters = getclusters(centroids)
    img.setflags(write=1)
    for i in range(k):
        img[np.where(clusters==i)] = centroids[i]
    image=Image.fromarray(img)
    image.show()
    return img

In [165]:
def dice_coef(y_true, y_pred,smooth=1):
    y_true_f = np.ndarray.flatten(y_true)
    y_pred_f = np.ndarray.flatten(y_pred)
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection ) / (np.sum(y_true_f) + np.sum(y_pred_f) )

In [166]:
for k in K:
    print(k)
    centroids = np.ndarray(shape = (k,3))
    old_centroids =  np.zeros(shape= (k,3))
    i=0
    while i<k:
        centroid = imgarr[np.random.randint(0,img_height),np.random.randint(0,img_width)]
        if centroid in centroids:
            continue
        centroids[i] = centroid
        i+=1

    j=0 
    while not converged(centroids,old_centroids) and j<=50:
        old_centroids= centroids
        clusters = getclusters(centroids)
        centroids = getcentroids(centroids,clusters)                              
        j+=1

    
predict = getsegmentation(centroids.astype(int))



26


In [167]:
dice_coef(imgarr/255.,imgarr/255.)

0.4725424975903439

In [116]:
a = np.array([1,2,3])

In [117]:
2*a*a/(a+a)

array([1., 2., 3.])

In [128]:
imgarr/255.

array([[[0.00392157, 0.00392157, 0.        ],
        [0.00392157, 0.00392157, 0.        ],
        [0.00392157, 0.00392157, 0.        ],
        ...,
        [0.        , 0.00392157, 0.        ],
        [0.        , 0.00392157, 0.        ],
        [0.        , 0.00392157, 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.00392157, 0.00392157, 0.        ],
        ...,
        [0.        , 0.00784314, 0.        ],
        [0.        , 0.00784314, 0.        ],
        [0.        , 0.00392157, 0.        ]],

       [[0.        , 0.00392157, 0.        ],
        [0.        , 0.00392157, 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.00392157, 0.00392157, 0.        ],
        [0.        , 0.00392157, 0.        ],
        [0.        , 0.00392157, 0.        ]],

       ...,

       [[0.        , 0.00392157, 0.        ],
        [0.        , 0.00392157, 0.        ],
        [0.        , 0